# Q2: Baseline Hồi Quy - Giải Thích & Phân Tích

Notebook này giải thích các khái niệm quan trọng trong baseline hồi quy cho dự báo PM2.5:

1. **Vì sao lag 24h có ý nghĩa?**
2. **Vì sao phải chia theo thời gian bằng cutoff?**
3. **Phân biệt RMSE và MAE**

## 1. Vì sao Lag 24h thường có ý nghĩa?

### Lý thuyết:

**Lag 24 giờ (1 ngày) có ý nghĩa vì:**

1. **Nhịp sinh hoạt theo ngày (Daily Activity Cycle):**
   - Hoạt động con người có chu kỳ 24h: giao thông, công nghiệp, sinh hoạt
   - Giờ cao điểm sáng/chiều → ô nhiễm tăng theo pattern
   - Đêm khuya → ít hoạt động → ô nhiễm giảm
   - Pattern này lặp lại mỗi ngày → PM2.5 hôm nay tương tự PM2.5 cùng giờ hôm qua

2. **Điều kiện khí tượng lặp lại theo chu kỳ ngày:**
   - Nhiệt độ, áp suất, độ ẩm có chu kỳ ngày-đêm
   - Hướng gió, tốc độ gió thường có pattern hàng ngày
   - Điều kiện khí tượng tương tự → khả năng khuếch tán ô nhiễm tương tự

3. **Hiện tượng PBL (Planetary Boundary Layer):**
   - Chiều cao lớp biên hành tinh thay đổi theo chu kỳ ngày
   - Ban ngày: PBL cao → ô nhiễm khuếch tán
   - Ban đêm: PBL thấp → ô nhiễm tập trung
   - Chu kỳ này lặp lại mỗi 24h

### Minh họa thực nghiệm:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

In [ ]:
# Load data
from src.classification_library import load_beijing_air_quality, clean_air_quality_df

RAW_ZIP_PATH = '../data/raw/PRSA2017_Data_20130301-20170228.zip'
df_raw = load_beijing_air_quality(use_ucimlrepo=False, raw_zip_path=RAW_ZIP_PATH)
df = clean_air_quality_df(df_raw)

# Chọn một trạm
STATION = 'Aotizhongxin'
df_station = df[df['station'] == STATION].sort_values('datetime').reset_index(drop=True)
print(f"Data shape: {df_station.shape}")

In [ ]:
# Tạo lag features
df_lag = df_station[['datetime', 'PM2.5']].copy()
df_lag['hour'] = df_lag['datetime'].dt.hour

# Tạo nhiều lag khác nhau
lags = [1, 6, 12, 24, 48, 72, 168]  # 1h, 6h, 12h, 1day, 2day, 3day, 1week
for lag in lags:
    df_lag[f'PM25_lag{lag}'] = df_lag['PM2.5'].shift(lag)

# Tính correlation
print("=" * 60)
print("CORRELATION PM2.5(t) với các LAG")
print("=" * 60)

correlations = {}
for lag in lags:
    corr = df_lag['PM2.5'].corr(df_lag[f'PM25_lag{lag}'])
    correlations[f'Lag {lag}h'] = corr
    print(f"Lag {lag:3d}h: {corr:.4f}")

# Visualize
fig, ax = plt.subplots(figsize=(10, 6))
lag_names = list(correlations.keys())
lag_values = list(correlations.values())
colors = ['red' if '24' in name else 'steelblue' for name in lag_names]
ax.bar(lag_names, lag_values, color=colors, alpha=0.7)
ax.set_ylabel('Correlation', fontsize=12)
ax.set_title('Correlation PM2.5(t) với các Lag khác nhau', fontsize=13, fontweight='bold')
ax.axhline(y=0.5, color='green', linestyle='--', alpha=0.5, label='Threshold 0.5')
ax.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\n⚠️ NHẬN XÉT:")
print(f"✅ Lag 24h có correlation = {correlations['Lag 24h']:.3f}")
print("→ Rất cao, cho thấy PM2.5 có chu kỳ ngày rõ ràng")
print("→ Lag 24h là feature QUAN TRỌNG NHẤT cho mô hình hồi quy")

In [ ]:
# Minh họa: PM2.5 theo giờ trong ngày
df_hourly = df_station.copy()
df_hourly['hour'] = df_hourly['datetime'].dt.hour

# Tính trung bình PM2.5 theo giờ
hourly_pattern = df_hourly.groupby('hour')['PM2.5'].agg(['mean', 'std']).reset_index()

fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(hourly_pattern['hour'], hourly_pattern['mean'], marker='o', linewidth=2, 
        markersize=8, color='darkblue', label='Mean PM2.5')
ax.fill_between(hourly_pattern['hour'], 
                 hourly_pattern['mean'] - hourly_pattern['std'],
                 hourly_pattern['mean'] + hourly_pattern['std'],
                 alpha=0.3, color='lightblue', label='±1 std')
ax.set_xlabel('Giờ trong ngày', fontsize=12)
ax.set_ylabel('PM2.5 (μg/m³)', fontsize=12)
ax.set_title('Pattern PM2.5 theo giờ trong ngày (Average)', fontsize=13, fontweight='bold')
ax.set_xticks(range(0, 24, 2))
ax.grid(True, alpha=0.3)
ax.legend()
plt.tight_layout()
plt.show()

print("\n⚠️ NHẬN XÉT:")
print("✅ PM2.5 có pattern rõ ràng theo giờ trong ngày")
print(f"   - Cao điểm: giờ {hourly_pattern.loc[hourly_pattern['mean'].idxmax(), 'hour']:.0f}h")
print(f"   - Thấp điểm: giờ {hourly_pattern.loc[hourly_pattern['mean'].idxmin(), 'hour']:.0f}h")
print("→ Đây là lý do vì sao lag 24h rất có ý nghĩa!")

## 2. Vì sao phải chia theo thời gian bằng CUTOFF?

### Lý thuyết:

**CUTOFF (temporal split) là THIẾT YẾU trong time series để:**

1. **Tránh Data Leakage (Rò rỉ dữ liệu):**
   - Nếu dùng random split → model có thể "nhìn thấy tương lai"
   - Ví dụ: train trên 2017-03-01, test trên 2017-02-15 → không thực tế!
   - Trong thực tế, chúng ta KHÔNG THỂ dùng dữ liệu tương lai để dự đoán quá khứ

2. **Đánh giá đúng khả năng generalization:**
   - Model cần dự đoán tốt trên dữ liệu CHƯA THẤY và ở TƯƠNG LAI
   - Random split → performance ảo, không phản ánh khả năng thực tế
   - Temporal split → performance thật, đánh giá đúng model

3. **Mô phỏng production environment:**
   - Khi deploy, model sẽ dự đoán tương lai dựa trên quá khứ
   - Train/test split phải giống với cách model được dùng thực tế

### Minh họa Data Leakage:

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Chuẩn bị data với lag features
df_model = df_station[['datetime', 'PM2.5', 'TEMP', 'PRES', 'DEWP', 'WSPM']].copy()
df_model['PM25_lag1'] = df_model['PM2.5'].shift(1)
df_model['PM25_lag24'] = df_model['PM2.5'].shift(24)
df_model['hour'] = df_model['datetime'].dt.hour
df_model['dayofweek'] = df_model['datetime'].dt.dayofweek
df_model = df_model.dropna()

feature_cols = ['PM25_lag1', 'PM25_lag24', 'TEMP', 'PRES', 'DEWP', 'WSPM', 'hour', 'dayofweek']
X = df_model[feature_cols]
y = df_model['PM2.5']

print("=" * 60)
print("SO SÁNH: RANDOM SPLIT vs TEMPORAL SPLIT")
print("=" * 60)

# Method 1: Random Split (SAI!)
X_train_rand, X_test_rand, y_train_rand, y_test_rand = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model_rand = RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42, n_jobs=-1)
model_rand.fit(X_train_rand, y_train_rand)
y_pred_rand = model_rand.predict(X_test_rand)

rmse_rand = np.sqrt(mean_squared_error(y_test_rand, y_pred_rand))
mae_rand = mean_absolute_error(y_test_rand, y_pred_rand)
r2_rand = r2_score(y_test_rand, y_pred_rand)

print("\n1. RANDOM SPLIT (❌ SAI - có leakage):")
print(f"   RMSE: {rmse_rand:.2f}")
print(f"   MAE:  {mae_rand:.2f}")
print(f"   R²:   {r2_rand:.4f}")

# Method 2: Temporal Split (ĐÚNG!)
cutoff_date = pd.Timestamp('2017-01-01')
train_mask = df_model['datetime'] < cutoff_date
test_mask = df_model['datetime'] >= cutoff_date

X_train_temp = X[train_mask]
X_test_temp = X[test_mask]
y_train_temp = y[train_mask]
y_test_temp = y[test_mask]

model_temp = RandomForestRegressor(n_estimators=50, max_depth=10, random_state=42, n_jobs=-1)
model_temp.fit(X_train_temp, y_train_temp)
y_pred_temp = model_temp.predict(X_test_temp)

rmse_temp = np.sqrt(mean_squared_error(y_test_temp, y_pred_temp))
mae_temp = mean_absolute_error(y_test_temp, y_pred_temp)
r2_temp = r2_score(y_test_temp, y_pred_temp)

print("\n2. TEMPORAL SPLIT (✅ ĐÚNG - không leakage):")
print(f"   RMSE: {rmse_temp:.2f}")
print(f"   MAE:  {mae_temp:.2f}")
print(f"   R²:   {r2_temp:.4f}")

print("\n" + "=" * 60)
print("PHÂN TÍCH SỰ KHÁC BIỆT")
print("=" * 60)
print(f"Chênh lệch RMSE: {abs(rmse_rand - rmse_temp):.2f}")
print(f"Chênh lệch MAE:  {abs(mae_rand - mae_temp):.2f}")
print(f"Chênh lệch R²:   {abs(r2_rand - r2_temp):.4f}")

if rmse_rand < rmse_temp:
    print("\n⚠️ KẾT LUẬN:")
    print("❌ Random split cho kết quả ĐẸP HƠN (RMSE thấp hơn)")
    print("   → Nhưng đây là kết quả GIẢ do leakage!")
    print("   → Model đã 'lén nhìn' tương lai trong quá trình train")
    print("   → Khi deploy thực tế, performance sẽ GIẢM như temporal split")
    print("\n✅ Temporal split cho performance THẬT, dù thấp hơn")
    print("   → Đây mới là khả năng thực sự của model")
    print("   → Nên LUÔN dùng temporal split cho time series!")

In [ ]:
# Visualize split methods
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Plot 1: Temporal split visualization
ax1 = axes[0]
ax1.scatter(df_model[train_mask]['datetime'], df_model[train_mask]['PM2.5'], 
            s=1, alpha=0.5, color='blue', label='Train')
ax1.scatter(df_model[test_mask]['datetime'], df_model[test_mask]['PM2.5'], 
            s=1, alpha=0.5, color='red', label='Test')
ax1.axvline(x=cutoff_date, color='green', linestyle='--', linewidth=2, label=f'Cutoff: {cutoff_date.date()}')
ax1.set_ylabel('PM2.5', fontsize=11)
ax1.set_title('✅ TEMPORAL SPLIT - Train trước cutoff, Test sau cutoff', fontsize=12, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Random split visualization (to show the problem)
ax2 = axes[1]
df_model_indexed = df_model.reset_index(drop=True)
train_indices_rand = X_train_rand.index
test_indices_rand = X_test_rand.index
ax2.scatter(df_model_indexed.loc[train_indices_rand, 'datetime'], 
            df_model_indexed.loc[train_indices_rand, 'PM2.5'], 
            s=1, alpha=0.5, color='blue', label='Train (scattered)')
ax2.scatter(df_model_indexed.loc[test_indices_rand, 'datetime'], 
            df_model_indexed.loc[test_indices_rand, 'PM2.5'], 
            s=1, alpha=0.5, color='red', label='Test (scattered)')
ax2.set_ylabel('PM2.5', fontsize=11)
ax2.set_xlabel('Time', fontsize=11)
ax2.set_title('❌ RANDOM SPLIT - Train/Test xen kẽ (có leakage!)', fontsize=12, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n⚠️ NHẬN XÉT:")
print("Với random split, train và test XEN KẼ nhau theo thời gian")
print("→ Model có thể học pattern từ 'tương lai' (test set) thông qua lag features")
print("→ Performance cao giả tạo, KHÔNG phản ánh khả năng dự báo thực tế")

## 3. Phân biệt RMSE và MAE

### Lý thuyết:

**MAE (Mean Absolute Error):**
- $MAE = \frac{1}{n}\sum_{i=1}^{n}|y_i - \hat{y}_i|$
- Trung bình của **trị tuyệt đối** sai số
- Đối xử **bình đẳng** với mọi sai số
- Dễ hiểu: "Trung bình model sai bao nhiêu?"

**RMSE (Root Mean Squared Error):**
- $RMSE = \sqrt{\frac{1}{n}\sum_{i=1}^{n}(y_i - \hat{y}_i)^2}$
- Căn bậc 2 của trung bình **bình phương** sai số
- **Phạt nặng** sai số lớn (do bình phương)
- Nhạy cảm với outliers và spike

**Khi nào RMSE cao hơn MAE nhiều?**
- Khi có **spike** (giá trị bất thường)
- Khi dự báo **sai mạnh** ở một số thời điểm
- Khi phân phối sai số **không đồng đều** (có vài điểm sai rất lớn)

### Minh họa:

In [ ]:
# Sử dụng predictions từ temporal split
errors = y_test_temp.values - y_pred_temp

mae_calc = np.mean(np.abs(errors))
rmse_calc = np.sqrt(np.mean(errors**2))

print("=" * 60)
print("PHÂN TÍCH RMSE vs MAE")
print("=" * 60)
print(f"MAE:  {mae_calc:.2f}  (trung bình sai số)")
print(f"RMSE: {rmse_calc:.2f}  (phạt nặng sai số lớn)")
print(f"RMSE/MAE ratio: {rmse_calc/mae_calc:.3f}")

print("\n⚠️ NHẬN XÉT:")
if rmse_calc / mae_calc > 1.5:
    print("❌ RMSE >> MAE → có nhiều sai số LỚN hoặc SPIKE")
    print("   → Model dự đoán SAI MẠNH ở một số thời điểm")
    print("   → Cần cải thiện model hoặc xử lý outliers")
elif rmse_calc / mae_calc > 1.2:
    print("⚠️ RMSE > MAE (bình thường) → có vài sai số lớn hơn")
    print("   → Phân phối sai số không hoàn toàn đồng đều")
else:
    print("✅ RMSE ≈ MAE → phân phối sai số khá đồng đều")
    print("   → Model dự đoán ổn định")

In [ ]:
# Visualize error distribution
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Error distribution histogram
ax1 = axes[0, 0]
ax1.hist(errors, bins=50, color='steelblue', alpha=0.7, edgecolor='black')
ax1.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Zero error')
ax1.axvline(x=mae_calc, color='green', linestyle='--', linewidth=2, label=f'MAE={mae_calc:.1f}')
ax1.axvline(x=-mae_calc, color='green', linestyle='--', linewidth=2)
ax1.set_xlabel('Error (actual - predicted)', fontsize=11)
ax1.set_ylabel('Frequency', fontsize=11)
ax1.set_title('Phân phối SAI SỐ', fontsize=12, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Absolute errors
ax2 = axes[0, 1]
abs_errors = np.abs(errors)
ax2.hist(abs_errors, bins=50, color='coral', alpha=0.7, edgecolor='black')
ax2.axvline(x=mae_calc, color='blue', linestyle='--', linewidth=2, label=f'MAE={mae_calc:.1f}')
ax2.axvline(x=rmse_calc, color='red', linestyle='--', linewidth=2, label=f'RMSE={rmse_calc:.1f}')
ax2.set_xlabel('|Error|', fontsize=11)
ax2.set_ylabel('Frequency', fontsize=11)
ax2.set_title('Phân phối SAI SỐ TUYỆT ĐỐI', fontsize=12, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Squared errors
ax3 = axes[1, 0]
squared_errors = errors**2
ax3.hist(squared_errors, bins=50, color='orange', alpha=0.7, edgecolor='black')
ax3.axvline(x=rmse_calc**2, color='red', linestyle='--', linewidth=2, 
            label=f'MSE={rmse_calc**2:.1f}')
ax3.set_xlabel('Error²', fontsize=11)
ax3.set_ylabel('Frequency', fontsize=11)
ax3.set_title('Phân phối SAI SỐ BÌNH PHƯƠNG (nhấn mạnh spike)', fontsize=12, fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Plot 4: Time series of absolute errors
ax4 = axes[1, 1]
test_dates = df_model[test_mask]['datetime'].values
ax4.plot(test_dates, abs_errors, linewidth=0.8, alpha=0.7, color='purple')
ax4.axhline(y=mae_calc, color='blue', linestyle='--', linewidth=2, label=f'MAE={mae_calc:.1f}')
ax4.axhline(y=rmse_calc, color='red', linestyle='--', linewidth=2, label=f'RMSE={rmse_calc:.1f}')
ax4.set_xlabel('Time', fontsize=11)
ax4.set_ylabel('|Error|', fontsize=11)
ax4.set_title('SAI SỐ theo THỜI GIAN', fontsize=12, fontweight='bold')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Phân tích: Tìm các điểm có sai số lớn
large_error_threshold = np.percentile(abs_errors, 95)  # top 5% sai số
large_error_mask = abs_errors > large_error_threshold
n_large_errors = large_error_mask.sum()

print("=" * 60)
print("PHÂN TÍCH SAI SỐ LỚN (top 5%)")
print("=" * 60)
print(f"Ngưỡng sai số lớn: {large_error_threshold:.2f}")
print(f"Số điểm có sai số lớn: {n_large_errors} / {len(abs_errors)} ({n_large_errors/len(abs_errors)*100:.1f}%)")
print(f"\nĐóng góp của top 5% sai số vào:")
print(f"  - MAE:  {np.mean(abs_errors[large_error_mask])/mae_calc:.2f}x")
print(f"  - MSE:  {np.mean(squared_errors[large_error_mask])/(rmse_calc**2):.2f}x")

print("\n⚠️ KẾT LUẬN về RMSE vs MAE:")
contribution_ratio = np.mean(squared_errors[large_error_mask])/(rmse_calc**2)
if contribution_ratio > 2:
    print("❌ Top 5% sai số LỚN đóng góp RẤT NHIỀU vào RMSE")
    print("   → RMSE cao hơn MAE đáng kể")
    print("   → Model có xu hướng dự báo SAI MẠNH ở một số spike/outlier")
    print("   → Nên tập trung cải thiện model tại các điểm này")
else:
    print("✅ Sai số phân bố tương đối đều")
    print("   → RMSE và MAE đều phản ánh tốt performance của model")

## 📝 TÓM TẮT Q2

### 1. Vì sao Lag 24h có ý nghĩa?
✅ **3 lý do chính:**
- Nhịp sinh hoạt con người có chu kỳ 24h (giao thông, công nghiệp)
- Điều kiện khí tượng lặp lại theo chu kỳ ngày-đêm
- Hiện tượng PBL (lớp biên hành tinh) thay đổi theo chu kỳ ngày

→ **PM2.5 cùng giờ hôm qua là predictor tốt nhất cho PM2.5 hôm nay**

### 2. Vì sao phải chia theo cutoff?
✅ **Tránh data leakage:**
- Random split → model "nhìn thấy tương lai" → performance giả
- Temporal split → model chỉ học từ quá khứ → performance thật
- Trong production, luôn dự đoán tương lai từ quá khứ → cần train/test giống vậy

→ **LUÔN dùng temporal split cho time series!**

### 3. RMSE vs MAE?
✅ **Sự khác biệt:**
- MAE: trung bình sai số, đối xử bình đẳng
- RMSE: phạt nặng sai số lớn (do bình phương)
- RMSE >> MAE → có spike/outlier, model dự báo sai mạnh ở vài điểm

→ **Nên báo cáo CẢ HAI metrics để hiểu đầy đủ performance!**